Querying data that will be used to train the neural network
---
1) querying all coins for which will build a correlated neural network model

2) keeping only the data of interest in an array:

    t-m:   p1 p2 p3 p4 ... pn
    t-m+1: p1 p2 p3 p4 ... pn
    t-m+2: p1 p2 p3 p4 ... pn 
    ...
    t:     p1 p2 p3 p4 ... pn

3) the vars array will have shape (n currencies, m points)

In [1]:
import numpy as np

from cryptocompare import get_histo

vars = []
mhd = 'hour'
candle_width = 1
m = 800
e = 'Bitfinex'
currencies = ['BTC', 'ETH', 'XRP', 'TRX', 'NEO']
n = len(currencies)
for coin in currencies:
    q = get_histo(mhd, coin, m, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        if d0['close'] == 0:
            p = 0
        else:
            p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)
print('vars.shape', vars.shape)

vars.shape (5, 800)


### Particioning blocks of data that will be used to build the neural network
---
1) determining the number of points for each block

2) building vector of inputs (xs) and outputs (ys)

3) classifying responses for each cryptocurrency using the following scheme (only 2 classes, I tried with more classes but the prediction gets much worse):

    0 -->     y < threshold
    1 -->     y >= threshold
    

In [2]:
block = 50
xs = []
ys = []
for i in range(vars.shape[1] - (block+1)):
    part = vars[:, i:i+(block+1)]
    xs.append(part[:, :-1])
    ys.append(part[:, -1])
xs = np.array(xs)
print('xs.shape', xs.shape)
def classify(y):
    threshold = 0.02
    if y < threshold:
        return 0
    else:
        return 1
    
classes = {}
for i, currency in enumerate(currencies):
    tmp = []
    for y in ys:
        tmp.append(classify(y[i]))
    classes[currency] = np.array(tmp)
    

xs.shape (749, 5, 50)


Shuffling samples for a better stochastic response
---

In [3]:
import random
rand_indices = np.arange(xs.shape[0])
random.shuffle(rand_indices)
xs = xs[rand_indices]
for k in classes:
    classes[k] = classes[k][rand_indices]
print('xs.shape', xs.shape)

xs.shape (749, 5, 50)


Training a simple Keras model
---

Reference: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [4]:
xs = xs.reshape(xs.shape[0], -1)
print('xs.shape', xs.shape)

xs.shape (749, 250)


In [5]:
from keras.models import Sequential
from keras.layers import Dense

models = {}
for k in currencies:
    Y = classes[k]
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=xs.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    out = model.fit(xs, Y, epochs=150, batch_size=10)
    models[k] = model

Using TensorFlow backend.


Epoch 1/150
749/749 [==============================] - 0s 511us/step - loss: 0.6651 - acc: 0.8652
Epoch 2/150
749/749 [==============================] - 0s 116us/step - loss: 0.5672 - acc: 0.9146
Epoch 3/150
749/749 [==============================] - 0s 115us/step - loss: 0.4212 - acc: 0.9146
Epoch 4/150
749/749 [==============================] - 0s 102us/step - loss: 0.3207 - acc: 0.9146
Epoch 5/150
749/749 [==============================] - 0s 103us/step - loss: 0.2873 - acc: 0.9146
Epoch 6/150
749/749 [==============================] - 0s 103us/step - loss: 0.2783 - acc: 0.9146
Epoch 7/150
749/749 [==============================] - 0s 100us/step - loss: 0.2720 - acc: 0.9146
Epoch 8/150
749/749 [==============================] - 0s 99us/step - loss: 0.2663 - acc: 0.9146
Epoch 9/150
749/749 [==============================] - 0s 96us/step - loss: 0.2598 - acc: 0.9159
Epoch 10/150
749/749 [==============================] - 0s 102us/step - loss: 0.2520 - acc: 0.9172
Epoch 11/150
749/749 

749/749 [==============================] - 0s 99us/step - loss: 0.0291 - acc: 0.9933
Epoch 85/150
749/749 [==============================] - 0s 94us/step - loss: 0.0282 - acc: 0.9933
Epoch 86/150
749/749 [==============================] - 0s 87us/step - loss: 0.0275 - acc: 0.9933
Epoch 87/150
749/749 [==============================] - 0s 84us/step - loss: 0.0266 - acc: 0.9933
Epoch 88/150
749/749 [==============================] - 0s 84us/step - loss: 0.0261 - acc: 0.9933
Epoch 89/150
749/749 [==============================] - 0s 95us/step - loss: 0.0250 - acc: 0.9933
Epoch 90/150
749/749 [==============================] - 0s 94us/step - loss: 0.0238 - acc: 0.9933
Epoch 91/150
749/749 [==============================] - 0s 92us/step - loss: 0.0230 - acc: 0.9947
Epoch 92/150
749/749 [==============================] - 0s 88us/step - loss: 0.0224 - acc: 0.9947
Epoch 93/150
749/749 [==============================] - 0s 90us/step - loss: 0.0214 - acc: 0.9947
Epoch 94/150
749/749 [===========

749/749 [==============================] - 0s 115us/step - loss: 0.1556 - acc: 0.9493
Epoch 17/150
749/749 [==============================] - 0s 115us/step - loss: 0.1491 - acc: 0.9519
Epoch 18/150
749/749 [==============================] - 0s 104us/step - loss: 0.1434 - acc: 0.9546
Epoch 19/150
749/749 [==============================] - 0s 90us/step - loss: 0.1389 - acc: 0.9559
Epoch 20/150
749/749 [==============================] - 0s 88us/step - loss: 0.1341 - acc: 0.9586
Epoch 21/150
749/749 [==============================] - 0s 69us/step - loss: 0.1293 - acc: 0.9613
Epoch 22/150
749/749 [==============================] - 0s 103us/step - loss: 0.1248 - acc: 0.9640
Epoch 23/150
749/749 [==============================] - 0s 87us/step - loss: 0.1211 - acc: 0.9666
Epoch 24/150
749/749 [==============================] - 0s 88us/step - loss: 0.1168 - acc: 0.9680
Epoch 25/150
749/749 [==============================] - 0s 90us/step - loss: 0.1130 - acc: 0.9680
Epoch 26/150
749/749 [=======

749/749 [==============================] - 0s 87us/step - loss: 0.0024 - acc: 1.0000
Epoch 100/150
749/749 [==============================] - 0s 84us/step - loss: 0.0023 - acc: 1.0000
Epoch 101/150
749/749 [==============================] - 0s 84us/step - loss: 0.0022 - acc: 1.0000
Epoch 102/150
749/749 [==============================] - 0s 84us/step - loss: 0.0020 - acc: 1.0000
Epoch 103/150
749/749 [==============================] - 0s 87us/step - loss: 0.0020 - acc: 1.0000
Epoch 104/150
749/749 [==============================] - 0s 85us/step - loss: 0.0018 - acc: 1.0000
Epoch 105/150
749/749 [==============================] - 0s 89us/step - loss: 0.0017 - acc: 1.0000
Epoch 106/150
749/749 [==============================] - 0s 88us/step - loss: 0.0017 - acc: 1.0000
Epoch 107/150
749/749 [==============================] - 0s 84us/step - loss: 0.0015 - acc: 1.0000
Epoch 108/150
749/749 [==============================] - 0s 86us/step - loss: 0.0015 - acc: 1.0000
Epoch 109/150
749/749 [=

...

749/749 [==============================] - 0s 94us/step - loss: 0.0015 - acc: 1.0000
Epoch 114/150
749/749 [==============================] - 0s 88us/step - loss: 0.0014 - acc: 1.0000
Epoch 115/150
749/749 [==============================] - 0s 87us/step - loss: 0.0013 - acc: 1.0000
Epoch 116/150
749/749 [==============================] - 0s 84us/step - loss: 0.0013 - acc: 1.0000
Epoch 117/150
749/749 [==============================] - 0s 86us/step - loss: 0.0012 - acc: 1.0000
Epoch 118/150
749/749 [==============================] - 0s 86us/step - loss: 0.0011 - acc: 1.0000
Epoch 119/150
749/749 [==============================] - 0s 88us/step - loss: 0.0011 - acc: 1.0000
Epoch 120/150
749/749 [==============================] - 0s 103us/step - loss: 0.0010 - acc: 1.0000
Epoch 121/150
749/749 [==============================] - 0s 90us/step - loss: 9.8108e-04 - acc: 1.0000
Epoch 122/150
749/749 [==============================] - 0s 87us/step - loss: 9.3576e-04 - acc: 1.0000
Epoch 123/150
7

Epoch 45/150
749/749 [==============================] - 0s 100us/step - loss: 0.1370 - acc: 0.9586
Epoch 46/150
749/749 [==============================] - 0s 106us/step - loss: 0.1309 - acc: 0.9573
Epoch 47/150
749/749 [==============================] - 0s 111us/step - loss: 0.1320 - acc: 0.9613
Epoch 48/150
749/749 [==============================] - 0s 100us/step - loss: 0.1247 - acc: 0.9599
Epoch 49/150
749/749 [==============================] - 0s 103us/step - loss: 0.1203 - acc: 0.9626
Epoch 50/150
749/749 [==============================] - 0s 90us/step - loss: 0.1187 - acc: 0.9640
Epoch 51/150
749/749 [==============================] - 0s 95us/step - loss: 0.1144 - acc: 0.9626
Epoch 52/150
749/749 [==============================] - 0s 92us/step - loss: 0.1112 - acc: 0.9706
Epoch 53/150
749/749 [==============================] - 0s 115us/step - loss: 0.1082 - acc: 0.9680
Epoch 54/150
749/749 [==============================] - 0s 111us/step - loss: 0.1088 - acc: 0.9706
Epoch 55/150


749/749 [==============================] - 0s 86us/step - loss: 0.0046 - acc: 0.9987
Epoch 129/150
749/749 [==============================] - 0s 86us/step - loss: 0.0043 - acc: 0.9987
Epoch 130/150
749/749 [==============================] - 0s 83us/step - loss: 0.0039 - acc: 1.0000
Epoch 131/150
749/749 [==============================] - 0s 86us/step - loss: 0.0037 - acc: 0.9987
Epoch 132/150
749/749 [==============================] - 0s 87us/step - loss: 0.0035 - acc: 1.0000
Epoch 133/150
749/749 [==============================] - 0s 88us/step - loss: 0.0033 - acc: 1.0000
Epoch 134/150
749/749 [==============================] - 0s 84us/step - loss: 0.0031 - acc: 1.0000
Epoch 135/150
749/749 [==============================] - 0s 87us/step - loss: 0.0028 - acc: 1.0000
Epoch 136/150
749/749 [==============================] - 0s 90us/step - loss: 0.0026 - acc: 1.0000
Epoch 137/150
749/749 [==============================] - 0s 90us/step - loss: 0.0023 - acc: 1.0000
Epoch 138/150
749/749 [=

749/749 [==============================] - 0s 106us/step - loss: 0.1224 - acc: 0.9666
Epoch 62/150
749/749 [==============================] - 0s 90us/step - loss: 0.1195 - acc: 0.9653
Epoch 63/150
749/749 [==============================] - ETA: 0s - loss: 0.6940 - acc: 0.800 - 0s 63us/step - loss: 0.1173 - acc: 0.9680
Epoch 64/150
749/749 [==============================] - 0s 88us/step - loss: 0.1134 - acc: 0.9720
Epoch 65/150
749/749 [==============================] - 0s 67us/step - loss: 0.1115 - acc: 0.9706
Epoch 66/150
749/749 [==============================] - 0s 104us/step - loss: 0.1107 - acc: 0.9680
Epoch 67/150
749/749 [==============================] - 0s 91us/step - loss: 0.1062 - acc: 0.9706
Epoch 68/150
749/749 [==============================] - 0s 102us/step - loss: 0.1026 - acc: 0.9720
Epoch 69/150
749/749 [==============================] - 0s 106us/step - loss: 0.1001 - acc: 0.9746
Epoch 70/150
749/749 [==============================] - 0s 100us/step - loss: 0.0975 - ac

749/749 [==============================] - 0s 99us/step - loss: 0.0031 - acc: 1.0000
Epoch 144/150
749/749 [==============================] - 0s 122us/step - loss: 0.0030 - acc: 1.0000
Epoch 145/150
749/749 [==============================] - 0s 118us/step - loss: 0.0029 - acc: 1.0000
Epoch 146/150
749/749 [==============================] - 0s 115us/step - loss: 0.0027 - acc: 1.0000
Epoch 147/150
749/749 [==============================] - 0s 98us/step - loss: 0.0026 - acc: 1.0000
Epoch 148/150
749/749 [==============================] - 0s 107us/step - loss: 0.0024 - acc: 1.0000
Epoch 149/150
749/749 [==============================] - 0s 95us/step - loss: 0.0023 - acc: 1.0000
Epoch 150/150
749/749 [==============================] - 0s 106us/step - loss: 0.0022 - acc: 1.0000


Evaluating the models for each currency
---

In [6]:
for k in currencies:
    scores = models[k].evaluate(xs, classes[k])
    print("Currency %s. %s: %.2f%%" % (k, models[k].metrics_names[1], scores[1]*100))    

749/749 [==============================] - 0s 110us/step
Currency BTC. acc: 100.00%
749/749 [==============================] - 0s 104us/step
Currency ETH. acc: 100.00%
749/749 [==============================] - 0s 102us/step
Currency XRP. acc: 100.00%
749/749 [==============================] - 0s 113us/step
Currency TRX. acc: 100.00%
749/749 [==============================] - 0s 102us/step
Currency NEO. acc: 100.00%


Testing the predictor
---

Note that 0 is when the candle is < threshold and 1 when candle is >= threshold

In [7]:
print('Prediction', np.round(models['BTC'].predict(xs)).astype(int).flatten())
print('Ref:', np.round(classes['BTC'], 1))

Prediction [0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1

Testing predictor with new queried data
---

In [13]:
vars = []
for coin in currencies:
    q = get_histo(mhd, coin, block, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        if d0['close'] == 0:
            p = 0
        else:
            p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)

for k in currencies:
    print('Prediction for currency %s' % k, np.round(models[k].predict(vars.flatten().reshape(1, -1)), 0))

vars.shape (5, 50)
Prediction for currency BTC [[ 0.]]
Prediction for currency ETH [[ 0.]]
Prediction for currency XRP [[ 0.]]
Prediction for currency TRX [[ 0.]]
Prediction for currency NEO [[ 0.]]
